In [ ]:
import pathlib
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from fetch import file_index

The `fetch` script will download an index of all the historical topo maps that USGS has and open it in a pandas dataframe.
The map index contains information about each map including the grid size, what states each map covers, and URLs where that map can be downloaded.

In [ ]:
file_index.keys()

There are over 100k maps so we'll want to pick an area.

In [ ]:
len(file_index)

We'll focus on the area around Juneau, Alaska.
The keys we're interested in are the map name and the grid size.
We only want the highest-resolution maps.

In [ ]:
contains_juneau = file_index["map_name"].str.contains("Juneau")
high_resolution = file_index["grid_size"].str.contains("7.5 X 7.5 Minute")
juneau_maps = file_index[contains_juneau & high_resolution]

There are only 17 maps, so this should be manageable.

In [ ]:
juneau_maps[["map_name", "aerial_photo_year", "geotiff_url"]]

In [ ]:
for url in juneau_maps["geotiff_url"]:
    command = f"curl -C - -O {url}"
    subprocess.run(command.split())

In [ ]:
filenames = list(pathlib.Path("./").glob("AK_Juneau*.tif"))

Let's look at the first map.
If I know geospatial data, they've used some unholy coordinate system.

In [ ]:
with rasterio.open(filenames[0], "r") as source:
    crs = source.crs
    bounds = source.bounds
    image = source.read()

*sigh*

In [ ]:
crs

In [ ]:
bounds

In [ ]:
fig, ax = plt.subplots()
extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
ax.imshow(np.dstack((image[0, :, :], image[1, :, :], image[2, :, :])), extent=extent);